In [2]:
from datetime import datetime
import pytz
import openpyxl


def get_data(output_file_path, start_row, end_row, start_timezone, tasks_timezone):
    try:
        workbook = openpyxl.load_workbook(output_file_path, data_only=True)
        output_sheet = workbook.active

        cr_number_col = get_column_index(output_sheet, "CR number")
        collection_col = get_column_index(output_sheet, "Collection")
        Start_Date_col = get_column_index(output_sheet, "Start Date")
        

        for row_num in range(start_row, end_row + 1):
            
            #if sheet does not contain collection in scehduled tasks
            if output_sheet.cell(row=row_num, column=2).value == "CRs created/left:":
                continue
      
            cr_number = output_sheet.cell(row=row_num, column=cr_number_col).value
            collection = output_sheet.cell(row=row_num, column=collection_col).value

            start_date = output_sheet.cell(row=row_num, column=Start_Date_col).value
            start_date_object = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
            start_date_localized = start_timezone.localize(start_date_object)
            start_date_sfd = start_date_localized.astimezone(tasks_timezone)
            formatted_start_date = start_date_sfd.strftime("%m/%d/%Y %H:%M")
       
            fill_tasks(input_file_path, cr_number, collection, formatted_start_date)

    except Exception as e:
        print(f"An error occurred: {e}")

def get_column_index(sheet, column_name):
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

def fill_tasks(input_file_path, cr_number, collection, formatted_start_date):

    try:

        workbook = openpyxl.load_workbook(input_file_path, data_only=True)
        worksheet = workbook.active

        # Get column indices
        group_name_col = None
        change_request_col = None
        planned_start_date_col = None

        for tasks_column_idx in range(1, worksheet.max_column + 1):
            column_value = worksheet.cell(row=1, column=tasks_column_idx).value
            if column_value:
                column_value_lower = column_value.lower()
                if column_value_lower == 'collection':
                    group_name_col = tasks_column_idx
                elif column_value_lower == 'cr number':
                    change_request_col = tasks_column_idx
                elif column_value_lower == 'start date':
                    planned_start_date_col = tasks_column_idx
        
        if change_request_col is None:
            print("Error: 'CR number' not found.")
            return
        elif group_name_col is None:
            print("Error: 'Collection' not found.")
            return
        elif planned_start_date_col is None:
            print("Error: 'Start Date' not found.")
            return
        
        # Update columns for the relevant rows
        for row_idx in range(2, worksheet.max_row + 1):
            group_name = worksheet.cell(row=row_idx, column=group_name_col).value
            if group_name and collection.lower() in group_name.lower():
                worksheet.cell(row=row_idx, column=change_request_col).value = cr_number
                worksheet.cell(row=row_idx, column=planned_start_date_col).value = formatted_start_date


        # Save the changes
        workbook.save(input_file_path)
    
    except Exception as e:
        print(f"An error occurred: {e}")
     

if __name__ == "__main__":
    
    start_row = int(input("Enter the row: "))
    end_row = int(input("Enter the row: "))

    start_timezone = pytz.timezone('Europe/London')
    tasks_timezone = pytz.timezone('America/Los_Angeles')

    links_file_path = r'C:\Users\pelyu\OneDrive\Рабочий стол\chg\git\linkspath.txt'

    with open(links_file_path, 'r', encoding='utf-8') as file:
    # Read the lines
        lines = file.readlines()

    output_file_path = lines[0].strip() if len(lines) > 0 else ''
    input_file_path = lines[2].strip() if len(lines) > 2 else ''
   

    get_data(output_file_path, start_row, end_row, start_timezone, tasks_timezone)
